In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 4s (66.7 kB/s)
Reading package li

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [5]:
vine_reviews_df = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

vine_reviews_20 = vine_reviews_df.filter(vine_reviews_df["total_votes"] >= 20)

vine_reviews_20.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          1|          147|        175|   N|                Y|
|          1|           14|         31|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          4|           31|         36|   N|                N|
|          2|           19|         34|   N|                Y|
|          5|           28|         31|   N|                N|
|          1|            4|         32|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [6]:
vine_helpful_votes = vine_reviews_20.filter(vine_reviews_20["helpful_votes"] / vine_reviews_20["total_votes"] >= 0.5)

vine_helpful_votes.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          1|          147|        175|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          4|           31|         36|   N|                N|
|          2|           19|         34|   N|                Y|
|          5|           28|         31|   N|                N|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [12]:
vine_paid_reviews = vine_helpful_votes.filter(vine_helpful_votes["vine"] == "Y")

vine_paid_reviews.show(10) 

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           56|         63|   Y|                N|
|          3|           23|         23|   Y|                N|
|          5|           19|         24|   Y|                N|
|          4|           22|         26|   Y|                N|
|          3|           20|         26|   Y|                N|
|          5|           46|         51|   Y|                N|
|          5|           22|         28|   Y|                N|
|          5|           21|         24|   Y|                N|
|          4|           21|         28|   Y|                N|
|          5|           17|         20|   Y|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [13]:
vine_unpaid_reviews = vine_helpful_votes.filter(vine_helpful_votes["vine"] == "N")

vine_unpaid_reviews.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          1|          147|        175|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          4|           31|         36|   N|                N|
|          2|           19|         34|   N|                Y|
|          5|           28|         31|   N|                N|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [14]:
# breakdown of paid reviews

vine_paid_reviews.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|                94|               94|                94|  94|               94|
|   mean| 4.202127659574468|54.59574468085106|61.787234042553195|null|             null|
| stddev|0.9791348741656414|65.26098459822538| 68.90976994895392|null|             null|
|    min|                 1|               14|                20|   Y|                N|
|    max|                 5|              347|               362|   Y|                N|
+-------+------------------+-----------------+------------------+----+-----------------+



In [15]:
# breakdown of unpaid reviews

vine_unpaid_reviews.describe().show()



+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             40471|             40471|             40471|40471|            40471|
|   mean|  3.34765634651973|47.428405524943784|55.891057794470115| null|             null|
| stddev|1.6418850112078023|117.53763370687005|127.40280622961905| null|             null|
|    min|                 1|                10|                20|    N|                N|
|    max|                 5|             10498|             10780|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



In [18]:
# get count of paid 5 star reviews

vine_paid_5star = vine_paid_reviews[vine_paid_reviews["star_rating"] == 5].count()

vine_paid_5star

48

In [19]:
# get count of total paid reviews

paid_review_count = vine_paid_reviews.count()

paid_review_count

94

In [20]:
# get % of paid 5 star reviews

paid_5star_percent = vine_paid_5star / paid_review_count * 100

paid_5star_percent

51.06382978723404

In [21]:
# get count of unpaid 5 star reviews

vine_unpaid_5star = vine_unpaid_reviews[vine_unpaid_reviews["star_rating"] == 5].count()

vine_unpaid_5star

15663

In [22]:
# get count of total unpaid reviews

unpaid_review_count = vine_unpaid_reviews.count()

unpaid_review_count

40471

In [23]:
# get % of unpaid 5 star reviews

unpaid_5star_percent = vine_unpaid_5star / unpaid_review_count * 100

unpaid_5star_percent

38.701786464381904